In [15]:
import csv
import os
from tqdm import tqdm
import math

import requests
from dotenv import load_dotenv

load_dotenv()

True

In [55]:
API_URL = "https://api.openai.com/v1/chat/completions"

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}",
}

bias_factor = 100

logit_biases = {"9642": bias_factor, "2822": bias_factor}  # Yes and No

In [70]:
def rank_quality(system, prompt, logit_bias=True, debug=False):
    params = {
        "model": "gpt-4-0125-preview",
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
        "temperature": 0.0,
        "max_tokens": 1,
        "logprobs": True,
        "logit_bias": logit_biases if logit_bias else None,
        # "top_logprobs": 3,
    }
    try:
        r = requests.post(API_URL, json=params, headers=headers, timeout=5)
    except requests.exceptions.Timeout:
        return None

    if debug:
        print(r.json())
    prob_dict = r.json()["choices"][0]["logprobs"]["content"][0]
    prob_token = prob_dict["token"]
    if prob_token not in ["Yes", "No"]:
        return None

    prob = math.exp(prob_dict["logprob"])
    if prob_dict["token"] == "No":
        prob = 1 - prob

    return prob

In [73]:
# system = """
# You are a judge for the Pulitzer Prize with decades of writing experience. If you would vote in favor of awarding a Pulitzer Prize to the text the user provides, respond with "Yes". Otherwise, respond with "No."
# """
system = """
You are the editor-in-chief of The New York Times with decades of writing experience. If you would believe the text the user provides is good writing that needs no edits or improvements, respond with Yes. Otherwise, respond with No.
"""

prompt = """
In the sun-dappled sands of Santa Monica, a spectacle unlike any other has captured the hearts of onlookers and netizens alike. A group of kittens, their fur soft and eyes wide with curiosity, has embarked on an extraordinary journey — learning to play beach volleyball under the guidance of advanced large language models. These AI systems, typically known for their prowess in analyzing big data, translating languages, and even writing poetry, have found an unexpected application in coaching these tiny athletes. Through a series of gentle prompts and encouragements delivered via speakers dotting the court, the kittens have begun to show an impressive grasp of the game, batting a lightweight ball back and forth with surprising agility. This delightful merging of cutting-edge technology and animal playfulness not only offers a glimpse into a future where human and AI collaborations can extend into realms of pure joy and amusement but also serves as a testament to the limitless potential of interdisciplinary innovation. As the kittens pounce and play, they are not merely entertaining an audience; they are pioneers on the frontier of technology and animal training, demonstrating that with a dash of creativity, the future is boundless and bright.
"""

rank_quality(system.strip(), prompt.strip(), logit_bias=True, debug=True)

{'id': 'chatcmpl-8tp3mvHar2lcxnj2Nyxn4uCEAeN9h', 'object': 'chat.completion', 'created': 1708315006, 'model': 'gpt-4-0125-preview', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'No'}, 'logprobs': {'content': [{'token': 'No', 'logprob': -0.1797447, 'bytes': [78, 111], 'top_logprobs': []}]}, 'finish_reason': 'length'}], 'usage': {'prompt_tokens': 286, 'completion_tokens': 1, 'total_tokens': 287}, 'system_fingerprint': 'fp_7ddde83880'}


0.1645165168807795